# Xata | Xata

> [Xata](https://xata.io)は、`PostgreSQL`と`Elasticsearch`を基盤としたサーバーレスデータプラットフォームです。データベースとの対話にPython SDKを提供し、データの管理にはUIを利用できます。`XataChatMessageHistory`クラスを使用することで、チャットセッションの長期的な保存にXataデータベースを活用することができます。
>
> > [Xata](https://xata.io) is a serverless data platform, based on `PostgreSQL` and `Elasticsearch`. It provides a Python SDK for interacting with your database, and a UI for managing your data. With the `XataChatMessageHistory` class, you can use Xata databases for longer-term persistence of chat sessions.

このノートブックには以下が含まれています：

> This notebook covers:

* `XataChatMessageHistory`が何をするのかを示す簡単な例。

  > A simple example showing what `XataChatMessageHistory` does.

* Xataのベクターストアに保存された知識ベースやドキュメントに基づいて質問に答えるREACTエージェントを使用した、より複雑な例を紹介します。また、このエージェントは過去のメッセージの長期にわたる検索可能な履歴をXataのメモリーストアに保持しているという特徴も持っています。

  > A more complex example using a REACT agent that answer questions based on a knowledge based or documentation (stored in Xata as a vector store) and also having a long-term searchable history of its past messages (stored in Xata as a memory store)





## Setup | セットアップ

### Create a database | データベースを作成する

[Xata UI](https://app.xata.io)で新しいデータベースを作成してください。名前は何でも構いませんが、このノートでは`langchain`という名前を使用します。Langchain統合機能は、メモリを格納するために使用されるテーブルを自動的に作成することができ、この例ではそれを利用します。テーブルを事前に作成したい場合は、適切なスキーマを確保し、クラスを作成する際に`create_table`を`False`に設定してください。テーブルを事前に作成すると、各セッションの初期化時にデータベースへの往復が1回省けます。

> In the [Xata UI](https://app.xata.io) create a new database. You can name it whatever you want, in this notepad we'll use `langchain`. The Langchain integration can auto-create the table used for storying the memory, and this is what we'll use in this example. If you want to pre-create the table, ensure it has the right schema and set `create_table` to `False` when creating the class. Pre-creating the table saves one round-trip to the database during each session initialization.




まず最初に、依存関係をインストールしましょう：

> Let's first install our dependencies:




In [ ]:
!pip install xata openai langchain

次に、Xataの環境変数を取得する必要があります。新しいAPIキーを作成するには、[アカウント設定](https://app.xata.io/settings)にアクセスしてください。データベースURLを見つけるには、作成したデータベースの設定ページに移動します。データベースURLは次のような形式になります：`https://demo-uni3q8.eu-west-1.xata.sh/db/langchain`。

> Next, we need to get the environment variables for Xata. You can create a new API key by visiting your [account settings](https://app.xata.io/settings). To find the database URL, go to the Settings page of the database that you have created. The database URL should look something like this: `https://demo-uni3q8.eu-west-1.xata.sh/db/langchain`.




In [ ]:
import getpass

api_key = getpass.getpass("Xata API key: ")
db_url = input("Xata database URL (copy it from your DB settings):")

## Create a simple memory store | シンプルなメモリストアを作成する

メモリストア機能を単独でテストするために、以下のコードスニペットを使用しましょう：

> To test the memory store functionality in isolation, let's use the following code snippet:




In [ ]:
from langchain.memory import XataChatMessageHistory

history = XataChatMessageHistory(
    session_id="session-1", api_key=api_key, db_url=db_url, table_name="memory"
)

history.add_user_message("hi!")

history.add_ai_message("whats up?")

上記のコードは、ID `session-1` のセッションを作成し、その中に2つのメッセージを保存します。上記のコードを実行した後、Xata UIにアクセスすると、`memory` という名前のテーブルが表示され、そこに2つのメッセージが追加されていることが確認できるはずです。

> The above code creates a session with the ID `session-1` and stores two messages in it. After running the above, if you visit the Xata UI, you should see a table named `memory` and the two messages added to it.

以下のコードを使用して、特定のセッションのメッセージ履歴を取得することができます：

> You can retrieve the message history for a particular session with the following code:




In [ ]:
history.messages

## Conversational Q\&A chain on your data with memory | あなたのデータを使った記憶機能付きの会話型Q\&Aチェーン

次に、OpenAI、Xata Vector Storeの統合、そしてXataメモリストアの統合を組み合わせて、フォローアップの質問と履歴機能を備えた、あなたのデータに基づくQ\&Aチャットボットを作成する、より複雑な例を見てみましょう。

> Let's now see a more complex example in which we combine OpenAI, the Xata Vector Store integration, and the Xata memory store integration to create a Q\&A chat bot on your data, with follow-up questions and history.




OpenAI APIにアクセスする必要があるので、APIキーを設定しましょう：

> We're going to need to access the OpenAI API, so let's configure the API key:




In [ ]:
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

チャットボットが回答を検索するドキュメントを格納するために、Xata UIを使用して`langchain`データベースに`docs`という名前のテーブルを追加し、以下のカラムを追加してください：

> To store the documents that the chatbot will search for answers, add a table named `docs` to your `langchain` database using the Xata UI, and add the following columns:

* `content`は「Text」タイプです。これは`Document.pageContent`の値を格納するために使用されます。

  > `content` of type "Text". This is used to store the `Document.pageContent` values.

* `embedding`は「Vector」タイプです。使用するモデルに合わせた次元数を使用してください。このノートブックでは、1536次元のOpenAIの埋め込みを使用しています。

  > `embedding` of type "Vector". Use the dimension used by the model you plan to use. In this notebook we use OpenAI embeddings, which have 1536 dimensions.





ベクトルストアを作成し、いくつかのサンプルドキュメントを追加しましょう：

> Let's create the vector store and add some sample docs to it:




In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.xata import XataVectorStore

embeddings = OpenAIEmbeddings()

texts = [
    "Xata is a Serverless Data platform based on PostgreSQL",
    "Xata offers a built-in vector type that can be used to store and query vectors",
    "Xata includes similarity search",
]

vector_store = XataVectorStore.from_texts(
    texts, embeddings, api_key=api_key, db_url=db_url, table_name="docs"
)

上記のコマンドを実行した後、Xata UIにアクセスすると、`docs`テーブルにドキュメントとそれらの埋め込みがロードされているのが確認できるはずです。

> After running the above command, if you go to the Xata UI, you should see the documents loaded together with their embeddings in the `docs` table.




それでは、ユーザーとAIの両方からのチャットメッセージを保存するためのConversationBufferMemoryを作成しましょう。

> Let's now create a ConversationBufferMemory to store the chat messages from both the user and the AI.




In [ ]:
from uuid import uuid4

from langchain.memory import ConversationBufferMemory

chat_memory = XataChatMessageHistory(
    session_id=str(uuid4()),  # needs to be unique per user session
    api_key=api_key,
    db_url=db_url,
    table_name="memory",
)
memory = ConversationBufferMemory(
    memory_key="chat_history", chat_memory=chat_memory, return_messages=True
)

これで、ベクトルストアとチャットメモリを一緒に使用するエージェントを作成する時が来ました。

> Now it's time to create an Agent to use both the vector store and the chat memory together.




In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.chat_models import ChatOpenAI

tool = create_retriever_tool(
    vector_store.as_retriever(),
    "search_docs",
    "Searches and returns documents from the Xata manual. Useful when you need to answer questions about Xata.",
)
tools = [tool]

llm = ChatOpenAI(temperature=0)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

テストとして、エージェントに私たちの名前を伝えましょう：

> To test, let's tell the agent our name:




In [ ]:
agent.run(input="My name is bob")

それでは、Xataについてエージェントにいくつか質問をしましょう：

> Now, let's now ask the agent some questions about Xata:




In [ ]:
agent.run(input="What is xata?")

ドキュメントストアに保存されているデータに基づいて回答することに注意してください。そして今、フォローアップの質問をしましょう：

> Notice that it answers based on the data stored in the document store. And now, let's ask a follow up question:




In [ ]:
agent.run(input="Does it support similarity search?")

それでは、その記憶力をテストしましょう：

> And now let's test its memory:




In [ ]:
agent.run(input="Did I tell you my name? What is it?")